Import

In [1]:
import sys
print("Python executable:", sys.executable)
print("Python version:", sys.version)

Python executable: /opt/miniconda3/envs/tld/bin/python
Python version: 3.8.20 | packaged by conda-forge | (default, Sep 30 2024, 17:48:42) 
[Clang 17.0.6 ]


In [7]:
import sys
import os
import glob
import torch

from tqdm import tqdm
from dotenv import load_dotenv
from ultralytics import YOLO

load_dotenv(verbose = False)
DIR_MODEL = os.getenv("DIR_MODEL")
sys.path.append("/Users/hunew/Desktop/Python_Files/Projects/Project_2(CV-Autonomous_driving)/EDA_test/ninja-turtles/src/machine/OD/ultralytics")



MODEL

In [10]:
def get_file_extension(filename):
    _, extension = os.path.splitext(filename)
    return extension.lstrip('.')


test_db_path = "/Users/hunew/Desktop/Python_Files/02_Storage/Project_2(CV-Autonomous_driving)/Data/test"
test_res_path = "%s/predictions"%(test_db_path)

if not os.path.exists(test_res_path):
    os.makedirs(test_res_path)

img_exts = ["jpg","bmp","png"]
img_files = list()
for img_ext in img_exts:
    img_files += glob.glob("%s/images/*.%s"%(test_db_path, img_ext))

img_files.sort() 

model_filename = "/Users/hunew/Desktop/Python_Files/02_Storage/Project_2(CV-Autonomous_driving)/Models/tld-Baseline/best.pt"
model = YOLO(model_filename)

# Check if MPS is available and set the device
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")


for img_filename in tqdm(img_files, desc="Processing images", unit="image", leave=True):
    result = model.predict(img_filename, imgsz=1280, conf=0.001, iou=0.6, device=device, verbose = False)[0]

    img_ext = get_file_extension(img_filename)
    txt_filename = img_filename.replace(img_ext, "txt")
    txt_filename = txt_filename.replace("images", "predictions")
    boxes = result.boxes 
    num_obj = len(boxes.cls)

    with open(txt_filename, 'w') as f1:
        for obj_idx in range(num_obj):
            cls_id = int(boxes.cls[obj_idx])
            cs = boxes.conf[obj_idx]
            xywhn = boxes.xywhn[obj_idx]
            # class_id norm_center_x norm_center_y norm_w norm_h confidence_score
            f1.write("%d %lf %lf %lf %lf %lf\n" % (cls_id, xywhn[0], xywhn[1], xywhn[2], xywhn[3], cs))

    if num_obj == 0:
        print(txt_filename)

Using device: mps


Processing images:  41%|████      | 5497/13502 [14:31<20:49,  6.41image/s]

/Users/hunew/Desktop/Python_Files/02_Storage/Project_2(CV-Autonomous_driving)/Data/test/predictions/10005495.txt


Processing images:  41%|████      | 5500/13502 [14:32<20:46,  6.42image/s]

/Users/hunew/Desktop/Python_Files/02_Storage/Project_2(CV-Autonomous_driving)/Data/test/predictions/10005498.txt


Processing images:  43%|████▎     | 5783/13502 [15:19<19:50,  6.48image/s]

/Users/hunew/Desktop/Python_Files/02_Storage/Project_2(CV-Autonomous_driving)/Data/test/predictions/10005781.txt


Processing images:  43%|████▎     | 5790/13502 [15:21<19:46,  6.50image/s]

/Users/hunew/Desktop/Python_Files/02_Storage/Project_2(CV-Autonomous_driving)/Data/test/predictions/10005788.txt


Processing images: 100%|██████████| 13502/13502 [37:08<00:00,  6.06image/s]
